In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"D:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 1000)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 103.80it/s]


Total cost = 39804.56957217159

Cluster 181 assigned to Class 7. Cost: 106.5685657975836
Cluster 237 assigned to Class 16. Cost: 98.33658482575518
Cluster 184 assigned to Class 18. Cost: 94.56041126440088
Cluster 362 assigned to Class 19. Cost: 124.47257297474957
Cluster 137 assigned to Class 21. Cost: 163.03583916910844
Cluster 259 assigned to Class 22. Cost: 112.64830615311415
Cluster 366 assigned to Class 23. Cost: 101.8561723144549
Cluster 130 assigned to Class 51. Cost: 123.12469616491462
Cluster 360 assigned to Class 87. Cost: 124.34283952618738
Cluster 164 assigned to Class 88. Cost: 122.56549174090395
Cluster 113 assigned to Class 89. Cost: 92.73851213748291
Cluster 124 assigned to Class 93. Cost: 77.77850584605494
Cluster 238 assigned to Class 95. Cost: 117.20928519491912
Cluster 69 assigned to Class 96. Cost: 78.7989402934782
Cluster 217 assigned to Class 105. Cost: 76.66948552385952
Cluster 294 assigned to Class 112. Cost: 128.34218972400558
Cluster 240 assigned to Class 117

In [5]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
